# Longformer for PubMed Articles 

Based on https://colab.research.google.com/github/patrickvonplaten/notebooks/blob/master/Fine_tune_Longformer_Encoder_Decoder_(LED)_for_Summarization_on_pubmed.ipynb 

In [ ]:
# crash colab to get more RAM
!kill -9 -1

In [2]:
# need this code1q snippet in each notebook to mount the google 
from google.colab import drive  
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [3]:
# Define directory to save checkpoints and outputs
result_dir = '/content/gdrive/MyDrive/w266_project/Baseline_v2'

In [4]:
# Install required packages
%%capture
!pip install datasets==1.2.1
!pip install transformers==4.2.0
!pip install rouge_score

In [5]:
# Load packages
from datasets import load_dataset, load_metric

In [6]:
# Download datasets
train_dataset = load_dataset("scientific_papers", "pubmed", split="train")

Dataset scientific_papers downloaded and prepared to /root/.cache/huggingface/datasets/scientific_papers/pubmed/1.1.1/043e40ed208b8a66ee9e8228c86874946c99d2fc6155a1daee685795851cfdfc. Subsequent calls will reuse this data.


In [7]:
val_dataset = load_dataset("scientific_papers", "pubmed", split="validation")

Reusing dataset scientific_papers (/root/.cache/huggingface/datasets/scientific_papers/pubmed/1.1.1/043e40ed208b8a66ee9e8228c86874946c99d2fc6155a1daee685795851cfdfc)


In [8]:
# Look at some data samples
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=4):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [9]:
show_random_elements(train_dataset, num_examples=1)

In [10]:
# Load package to tokenize
from transformers import AutoTokenizer

In [11]:
# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("allenai/led-base-16384")
# tokenizer = AutoTokenizer.from_pretrained("allenai/led-large-16384")

In [12]:
# Define high level parameters
max_input_length = 8192
max_output_length = 512
batch_size = 2

In [13]:
# Create function to tokenize Inputs and Labels
def process_data_to_model_inputs(batch):
    inputs = tokenizer(
        batch["article"],
        padding="max_length",
        truncation=True,
        max_length=max_input_length,
    )
    outputs = tokenizer(
        batch["abstract"],
        padding="max_length",
        truncation=True,
        max_length=max_output_length,
    )

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    # create 0 global_attention_mask lists
    batch["global_attention_mask"] = len(batch["input_ids"]) * [
        [0 for _ in range(len(batch["input_ids"][0]))]
    ]

    # since above lists are references, the following line changes the 0 index for all samples
    batch["global_attention_mask"][0][0] = 1
    batch["labels"] = outputs.input_ids

    # We have to make sure that the PAD token is ignored
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]

    return batch

In [14]:
# Reduce dataset sizes to run more quickly
train_dataset = train_dataset.select(range(10000))
val_dataset = val_dataset.select(range(1000))

# len(train_dataset)
# len(val_dataset)

In [15]:
# Process training dataset
train_dataset = train_dataset.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["article", "abstract", "section_names"],
)

In [16]:
# Process validation dataset
val_dataset = val_dataset.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["article", "abstract", "section_names"],
)

In [17]:
# Convert o pytorch
train_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
)
val_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
)

In [18]:
# Load packages
from transformers import AutoModelForSeq2SeqLM

In [19]:
# Load pre-trained model
led = AutoModelForSeq2SeqLM.from_pretrained("allenai/led-base-16384", gradient_checkpointing=True, use_cache=False)
# led = AutoModelForSeq2SeqLM.from_pretrained("allenai/led-large-16384", gradient_checkpointing=True, use_cache=False)

In [20]:
# Establish hyperparameters
led.config.num_beams = 2
led.config.max_length = 512
led.config.min_length = 100
led.config.length_penalty = 2.0
led.config.early_stopping = True
led.config.no_repeat_ngram_size = 3

In [21]:
# Load metric
rouge = load_metric("rouge")

In [22]:
# Function to compute Rouge score
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge1_output = rouge.compute(
        predictions=pred_str, references=label_str, rouge_types=["rouge1"]
    )["rouge1"].mid

    rouge2_output = rouge.compute(
        predictions=pred_str, references=label_str, rouge_types=["rouge2"]
    )["rouge2"].mid

    rouge3_output = rouge.compute(
        predictions=pred_str, references=label_str, rouge_types=["rouge3"]
    )["rouge3"].mid

    rougeL_output = rouge.compute(
        predictions=pred_str, references=label_str, rouge_types=["rougeL"]
    )["rougeL"].mid

    return {
        "rouge1_precision": round(rouge1_output.precision, 4),
        "rouge1_recall": round(rouge1_output.recall, 4),
        "rouge1_fmeasure": round(rouge1_output.fmeasure, 4),        

        "rouge2_precision": round(rouge2_output.precision, 4),
        "rouge2_recall": round(rouge2_output.recall, 4),
        "rouge2_fmeasure": round(rouge2_output.fmeasure, 4),

        "rouge3_precision": round(rouge3_output.precision, 4),
        "rouge3_recall": round(rouge3_output.recall, 4),
        "rouge3_fmeasure": round(rouge3_output.fmeasure, 4),        

        "rougeL_precision": round(rougeL_output.precision, 4),
        "rougeL_recall": round(rougeL_output.recall, 4),
        "rougeL_fmeasure": round(rougeL_output.fmeasure, 4),        
    }

In [23]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In [24]:
# Enable fp16 apex training
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True,
    # output_dir="./",
    output_dir=result_dir,
    logging_steps=5,
    eval_steps=10,
    save_steps=10,
    save_total_limit=2,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
)

In [25]:
# Pass arguments to trainer
trainer = Seq2SeqTrainer(
    model=led,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [ ]:
# Start training
trainer.train()

Step,Training Loss,Validation Loss,Rouge1 Precision,Rouge1 Recall,Rouge1 Fmeasure,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure,Rouge3 Precision,Rouge3 Recall,Rouge3 Fmeasure,Rougel Precision,Rougel Recall,Rougel Fmeasure,Runtime,Samples Per Second
10,2.922000,2.646711,0.423900,0.316500,0.324900,0.133400,0.100300,0.101900,0.060100,0.045400,0.046200,0.240900,0.179800,0.183600,12443.290600,0.080000
20,2.536800,2.403752,0.503700,0.265600,0.328900,0.174200,0.089300,0.111400,0.082100,0.042500,0.052600,0.294100,0.155500,0.192000,5546.166300,0.180000
30,2.611000,2.332934,0.519000,0.262700,0.330400,0.195400,0.096100,0.122100,0.103300,0.050600,0.064200,0.310200,0.156300,0.196500,5087.731400,0.197000
40,2.363300,2.275810,0.499900,0.287800,0.344400,0.182000,0.102500,0.123500,0.096100,0.053900,0.064800,0.292500,0.168100,0.200700,6688.116600,0.150000
50,2.319700,2.288058,0.500800,0.268800,0.331800,0.187000,0.098100,0.122200,0.103000,0.054100,0.067300,0.298900,0.160800,0.197900,5498.673500,0.182000
60,2.237700,2.232923,0.508000,0.278100,0.340100,0.192100,0.103300,0.127200,0.104300,0.056000,0.068900,0.300700,0.164200,0.200700,5701.383000,0.175000
70,2.221000,2.215832,0.510800,0.269200,0.335000,0.195500,0.100200,0.126100,0.109000,0.055100,0.069600,0.307200,0.161300,0.200600,5547.782800,0.180000
80,2.229600,2.213260,0.513200,0.261200,0.328600,0.192700,0.095600,0.121500,0.103800,0.051300,0.065200,0.308500,0.157000,0.197300,5171.735500,0.193000
90,2.326300,2.181675,0.505900,0.278300,0.341100,0.192100,0.103400,0.128000,0.102700,0.054900,0.068100,0.299700,0.165000,0.201800,6346.491400,0.158000


/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:911: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  self.args.max_grad_norm,
/usr/local/lib/python3.7/dist-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for 

Step,Training Loss,Validation Loss,Rouge1 Precision,Rouge1 Recall,Rouge1 Fmeasure,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure,Rouge3 Precision,Rouge3 Recall,Rouge3 Fmeasure,Rougel Precision,Rougel Recall,Rougel Fmeasure,Runtime,Samples Per Second
10,2.922000,2.646711,0.423900,0.316500,0.324900,0.133400,0.100300,0.101900,0.060100,0.045400,0.046200,0.240900,0.179800,0.183600,12443.290600,0.080000
20,2.536800,2.403752,0.503700,0.265600,0.328900,0.174200,0.089300,0.111400,0.082100,0.042500,0.052600,0.294100,0.155500,0.192000,5546.166300,0.180000
30,2.611000,2.332934,0.519000,0.262700,0.330400,0.195400,0.096100,0.122100,0.103300,0.050600,0.064200,0.310200,0.156300,0.196500,5087.731400,0.197000
40,2.363300,2.275810,0.499900,0.287800,0.344400,0.182000,0.102500,0.123500,0.096100,0.053900,0.064800,0.292500,0.168100,0.200700,6688.116600,0.150000
50,2.319700,2.288058,0.500800,0.268800,0.331800,0.187000,0.098100,0.122200,0.103000,0.054100,0.067300,0.298900,0.160800,0.197900,5498.673500,0.182000
60,2.237700,2.232923,0.508000,0.278100,0.340100,0.192100,0.103300,0.127200,0.104300,0.056000,0.068900,0.300700,0.164200,0.200700,5701.383000,0.175000
70,2.221000,2.215832,0.510800,0.269200,0.335000,0.195500,0.100200,0.126100,0.109000,0.055100,0.069600,0.307200,0.161300,0.200600,5547.782800,0.180000
80,2.229600,2.213260,0.513200,0.261200,0.328600,0.192700,0.095600,0.121500,0.103800,0.051300,0.065200,0.308500,0.157000,0.197300,5171.735500,0.193000
90,2.326300,2.181675,0.505900,0.278300,0.341100,0.192100,0.103400,0.128000,0.102700,0.054900,0.068100,0.299700,0.165000,0.201800,6346.491400,0.158000
100,2.399900,2.160119,0.505100,0.302300,0.359200,0.193400,0.113300,0.135600,0.103400,0.060100,0.072200,0.296300,0.177400,0.210200,7200.139800,0.139000


/usr/local/lib/python3.7/dist-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
